In [ ]:
# !pip install tensorflow-gpu
import os, cv2
from tensorflow import keras
from tensorflow.keras import Sequential
import numpy as np

In [ ]:
# 데이터 생성
'''cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")

capture = cv2.VideoCapture(0)
capture.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)'''


In [ ]:
'''while cv2.waitKey(33) < 0:
    ret, frame = capture.read()
    if ret:
        
        gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        result = cascade.detectMultiScale(
            gray, scaleFactor=1.1, minNeighbors=10, minSize=(20, 20))
            
        for x, y, w, h in result:

            print(x, y, x+w, y+h)
            date = datetime.strftime(datetime.now(), "%y%m%d-%H%M%S")
            cv2.imwrite(f"./user1/{date}.png", frame[y:y+h, x:x+h])
            cv2.rectangle(frame, (x, y), (x + w, y + h),
                          color=(255, 255, 0), thickness=1)
                          
        cv2.imshow("Video Frame", frame) # You can delete this to disable the window from opening 

capture.release()
cv2.destroyAllWindows()'''


In [ ]:

cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")
lists = os.listdir("others")

for arg in lists:
    dirs = "others/"+arg
    is_cropped = False
    frame = cv2.imread(dirs)
    gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    result = cascade.detectMultiScale(
        gray, scaleFactor=1.1, minNeighbors=10, minSize=(20, 20))
        
    for x, y, w, h in result:
        is_cropped = True
        cv2.imwrite(dirs, frame[y:y+h, x:x+h])
    if not is_cropped:
        os.remove(dirs)
print(len(os.listdir("others")))

In [ ]:
# LAG 에서 사진 가져오기
import shutil

main = "imgset"
target = "others"
names = os.listdir(main)

pictures = list()

for name in names:

    files = os.listdir(f"{main}/{name}")
    pictures += [f"{main}/{name}/{pic}" for pic in files if ".png" in pic]

    if "y" in files:
        pictures += [f"{main}/{name}/y/{pic}" for pic in os.listdir(
            f"{main}/{name}/y")]

for num, pictures in enumerate(pictures):

    shutil.move(pictures, f"{target}/arg_{num}.png")


In [ ]:
import random as r
classes = ["user1", "others"]

for dirs in classes:
    os.makedirs(f"trainset/{dirs}", exist_ok=True)
    os.makedirs(f"dataset/{dirs}", exist_ok=True)
    data = os.listdir(dirs)
    trainset = r.sample(data, k=400)
    data = [i for i in data if i not in trainset]
    dataset = r.sample(data, k=200)  # for datas = [i for i in datas if i not in trainset] --> 

    for train in trainset:
        shutil.copy(f"{dirs}/{train}", f"trainset/{dirs}")

    for test in dataset:
        shutil.copy(f"{dirs}/{test}", f"dataset/{dirs}")


In [ ]:
# Test network with random images
'''
import random as r

os.makedirs("trainset/user1", exist_ok=True)
trainset = r.sample(os.listdir("user1"), k=400)
dataset = r.sample(os.listdir("user1"), k=200)

for train in trainset:
    shutil.copy("user1/{train}", "trainset/user1")

for test in dataset:
    shutil.copy("user1/{test}", "dataset/user1")

os.makedirs("dataset/others", exist_ok=True)
trainset = r.sample(os.listdir("others"), k=400)
dataset = r.sample(os.listdir("others"), k=200)


for test in dataset:
    shutil.copy("{dirs}/{train}", "dataset/{dirs}")'''


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True
)
test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
    'trainset',
    target_size=(200, 200),
    batch_size=12,
    class_mode='binary'
)
test_set = test_datagen.flow_from_directory(
    'dataset',
    target_size=(200, 200),
    batch_size=12,
    class_mode='binary'
)


In [ ]:
from tensorflow.keras import layers
model = Sequential()
model.add(layers.Convolution2D(
    64, (3, 3), input_shape=(200, 200, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Convolution2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Convolution2D(16, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(3, 3)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.2))
model.add(layers.Dense(units=4096, activation='leaky_relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(units=1024, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(units=128, activation='relu'))
model.add(layers.Dense(units=2, activation='softmax'))
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
history = model.fit(training_set, steps_per_epoch=5, epochs=5,
                    validation_data=test_set, validation_steps=10)


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(16, 9))

for key, value in history.history.items():
    plt.plot(value, label=key)

plt.legend(loc='upper left')
plt.show()


In [ ]:
from tensorflow.keras.preprocessing import image


def predicter_dir(dtr):
    test = image.load_img(dtr, target_size=(200, 200, 3))
    test = image.img_to_array(test)
    test = np.expand_dims(test, axis=0)
    result = model.predict(test)
    indices = list(training_set.class_indices.keys())
    return indices[np.argmax(result[0])]


def predicter(pred):

    pred = cv2.resize(pred, dsize=(200, 200), interpolation=cv2.INTER_AREA)
    test = np.expand_dims(pred, axis=0)
    result = model.predict(test)
    indices = list(training_set.class_indices.keys())

    return indices[np.argmax(result[0])]


In [ ]:
from tensorflow.keras.models import save_model

save_model(model, "face.h5")


In [ ]:
print("test:user1")
for pic in os.listdir("user1")[350:]:
    if (result:=predicter_dir("user1/"+pic)) == "others":
        print(result, pic, end=", ")

print("test:others")
for pic in os.listdir("others")[:350]:
    if (result:=predicter_dir("others/"+pic)) == "user1":
        print(result, pic, end=", ")

        

In [ ]:
cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")
capture = cv2.VideoCapture(0)
capture.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
while cv2.waitKey(33) < 0:
    ret, frame = capture.read()
    if ret:
        gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        result = cascade.detectMultiScale(
            gray, scaleFactor=1.1, minNeighbors=10, minSize=(20, 20))
        for x, y, w, h in result:
            cut = frame[y:y+h, x:x+w]
            text = predicter(cut)
            frame = cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 255, 0), 2)
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(frame, text, (x, y), font, 1, (255, 255, 0), 2)
        cv2.imshow("Video Frame", frame)
capture.release()
cv2.destroyAllWindows()
